In [1]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from typing import Dict, List
from transformers import AutoTokenizer
from tqdm import tqdm
import scml
from scml import nlp as snlp
import mylib

In [2]:
percentiles = [.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()

In [3]:
pretrained_dir = "pretrained/google/electra-small-discriminator"
model_max_length = 512
tokenizer = AutoTokenizer.from_pretrained(pretrained_dir, model_max_length=model_max_length)
print(f"{repr(tokenizer)}\n{tokenizer.model_input_names}")

PreTrainedTokenizerFast(name_or_path='pretrained/google/electra-small-discriminator', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
['input_ids', 'token_type_ids', 'attention_mask']


In [4]:
train = pd.read_csv("input/train.csv", engine="c", low_memory=False)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19661 entries, 0 to 19660
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Id             19661 non-null  object
 1   pub_title      19661 non-null  object
 2   dataset_title  19661 non-null  object
 3   dataset_label  19661 non-null  object
 4   cleaned_label  19661 non-null  object
dtypes: object(5)
memory usage: 768.1+ KB


In [5]:
gt_map: Dict[str, List[str]] = {}
dl_map: Dict[str, List[str]] = {}
for t in tqdm(train.itertuples()):
    pid = getattr(t, "Id")
    cl = getattr(t, "cleaned_label")
    dl = getattr(t, "dataset_label")
    if pid not in gt_map:
        gt_map[pid] = list()
    if pid not in dl_map:
        dl_map[pid] = list()
    gt_map[pid].append(cl)
    dl_map[pid].append(dl)

19661it [00:00, 149757.67it/s]


In [6]:
rows = []
sep = "|"
m = 650
negative_examples = 3
max_length = 500
step = 50
for rid, gts in tqdm(gt_map.items()):
    is_multi = 0
    if len(gts) > 1:
        is_multi = 1
    ground_truth = sep.join(sorted(gts))
    dataset_labels = sep.join(sorted(dl_map[rid]))
    with open(f"input/train/{rid}.json") as in_file:
        sections = json.load(in_file)
    tmp = []
    for section in sections:
        tmp.append(section["text"])
    text = " ".join(tmp).strip()
    text = snlp.to_ascii_str(text)
    spans = []
    for dl in dl_map[rid]:
        i = 0
        j = len(text)
        while i < j - len(dl) + 1:
            p = text.find(dl, i, j)
            if p == -1:
                break
            q = p + len(dl)
            spans.append((p, q))
            i = q
            # Right context: answer is on the left edge of window
            a = p
            b = 2 * m + a
            _len = len(tokenizer(text[a:b])["input_ids"])
            while _len > max_length and a < b:
                b -= step * 2
                _len = len(tokenizer(text[a:b])["input_ids"])
            row = {}
            row["Id"] = rid
            row["is_multi"] = is_multi
            row["ground_truth"] = ground_truth
            row["dataset_labels"] = dataset_labels
            row["is_impossible"] = 0
            row["answer_start"] = 0
            row["answer_end"] = len(dl)
            row["context"] = text[a:b]
            row["context_token_length"] = _len
            rows.append(row)
            # Left context: answer is on the right edge of window
            b = q
            a = max(0, b - (2 * m))
            _len = len(tokenizer(text[a:b])["input_ids"])
            while _len > max_length and a < b:
                a += step * 2
                _len = len(tokenizer(text[a:b])["input_ids"])
            row = {}
            row["Id"] = rid
            row["is_multi"] = is_multi
            row["ground_truth"] = ground_truth
            row["dataset_labels"] = dataset_labels
            row["is_impossible"] = 0
            row["answer_end"] = b - a
            row["answer_start"] = row["answer_end"] - len(dl) 
            row["context"] = text[a:b]
            row["context_token_length"] = _len
            rows.append(row)
            # Left and right context: answer is in the middle of window
            a = max(0, p - m)
            b = q + m
            _len = len(tokenizer(text[a:b])["input_ids"])
            while _len > max_length and a < b:
                a += step
                b -= step
                _len = len(tokenizer(text[a:b])["input_ids"])
            row = {}
            row["Id"] = rid
            row["is_multi"] = is_multi
            row["ground_truth"] = ground_truth
            row["dataset_labels"] = dataset_labels
            row["is_impossible"] = 0
            row["context"] = text[a:b]
            row["answer_start"] = row["context"].index(dl)
            row["answer_end"] = row["answer_start"] + len(dl)
            row["context_token_length"] = _len
            rows.append(row)

100%|██████████| 14316/14316 [02:18<00:00, 103.69it/s]


In [7]:
df = pd.DataFrame.from_records(rows)
cols = ["is_multi", "is_impossible"]
df[cols] = df[cols].astype(np.int8)
cols = ["answer_start", "answer_end", "context_token_length"]
df[cols] = df[cols].astype(np.int16) 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183006 entries, 0 to 183005
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   Id                    183006 non-null  object
 1   is_multi              183006 non-null  int8  
 2   ground_truth          183006 non-null  object
 3   dataset_labels        183006 non-null  object
 4   is_impossible         183006 non-null  int8  
 5   answer_start          183006 non-null  int16 
 6   answer_end            183006 non-null  int16 
 7   context               183006 non-null  object
 8   context_token_length  183006 non-null  int16 
dtypes: int16(3), int8(2), object(4)
memory usage: 7.0+ MB


In [8]:
df["is_multi"].value_counts(normalize=True)

1    0.663519
0    0.336481
Name: is_multi, dtype: float64

In [9]:
assert (df["context_token_length"] <= max_length).all()
df["context_token_length"].describe(percentiles=percentiles)

count    183006.000000
mean        281.328645
std          47.780126
min           4.000000
1%           73.000000
5%          227.000000
10%         244.000000
20%         258.000000
30%         267.000000
40%         274.000000
50%         282.000000
60%         290.000000
70%         299.000000
80%         311.000000
90%         328.000000
95%         345.000000
99%         393.000000
max         500.000000
Name: context_token_length, dtype: float64

In [10]:
df.sample(20).head(20)

,Id,is_multi,ground_truth,dataset_labels,is_impossible,answer_start,answer_end,context,context_token_length
50237,dce3207b-8aac-4b80-8f9b-2657feb1a810,1,adni|alzheimer s disease neuroimaging initiative adni,ADNI|Alzheimer's Disease Neuroimaging Initiative (ADNI),0,650,654,"her the height scores calculated using the 2910 SNPs were correlated with the self-reported heights (at age 18) for the 407 individuals for which we have both height and genetic data. We failed to detect significant correlation between the two (correlation coefficient = 0.06, p = 0 25; Figure 3 ). This is consistent with the findings of the GIANT consortium. With a population of 1914 individuals, Wood et al. found a predictive r 2 = 0 14 for 697 SNPs (20% variation explained) [10] . It is expected that this r 2 should be stronger than the correlation coefficient in our findings because of our smaller population size of 407 individuals of the ADNI and Cache County individuals as well as the fact that the GIANT consortium identified the 697 SNPs used for prediction directly from their population of 1914 individuals. While research has shown that height is a polygenic trait heavily influenced by common SNPs [7] [8] [9] [10] [11] [12] , a polygenic score that quantifies common SNP effect is generally insufficient for successful individual phenotype prediction. We demonstrate that in the case of Mr. Bradley, a rare combination of common SNPs corresponds to an extremely high polygenic score that predicts an extreme phenotype. Because Mr. Bradley is an outlier, studying his genetic makeup p",285
9838,a5d51418-49db-49b5-ba2d-60b1b73419d9,1,baltimore longitudinal study of aging|baltimore longitudinal study of aging blsa,Baltimore Longitudinal Study of Aging|Baltimore Longitudinal Study of Aging (BLSA),0,1263,1300,"rt of this prototyping paradigm is the ""instrumentation"" (4.) window. When we consider a prototype from the engineering perspective, it seems natural that the capability exists to extract performance information from the executing program, whether it is considered an experiment, as in [I] and [9], or a model. Observing the output at this window, the designer is able to observe and analyze data extracted from the executing prototype, including operational parameters, performance indicators, and whatever other instrumentation is incorporated into the prototype, and track the progress of the prototype. The second part of our hypothesis concerns the flexibility of the model and the ways in which RBP can benefit the computer software development process. Consideration of this issue is the area in which our work has the potential to make a contribution to the body of knowledge in computer science. Our goal in this part of our project is to provide new insights into prototyping and reconfiguration that can be used as reference material for further experimentation with the RBP approach.\nCentral to our contention that prototyping can be more effectively undertaken using our model is the presentation of a case-in-point. We will draw an example from the Baltimore Longitudinal Study of Aging",246
770,cd1523cc-178d-4498-a35b-bbf53fa4fe6e,1,education longitudinal study|national education longitudinal study,Education Longitudinal Study|National Education Longitudinal Study,0,650,678,"gration to community colleges, with Latinos attending because of the presence of their friends and family (Perez & McDonough, 2008; Person & Rosenbaum, 2006) .\nAnother theme in the literature is that Latinos go to community colleges because they lack academic preparation. Although school-age Latinos have had lower reading and math scores compared with other students, they have been making steady gains in terms of grade point averages, standardized test scores, and high school credits earned (Llagas & Snyder, 2003) . However, academic credentials that make one eligible for a 4-year college do not automatically lead to enrollment. The National Education Longitudinal Study (NELS:88; National

In [11]:
df.to_parquet("output/train.parquet", index=False)